## Script to convert xyz to pdb using the [atomic simulation environment](https://ase-lib.org/index.html)
Install with: `pip install --upgrade ase`

In [1]:
import numpy as np
import glob
from ase.io import read, write

In [31]:
for file in glob.glob('structures/*.xyz'):
    print (f"Reading from file {file}:")
    atoms = read(file)
    non_h_indices = [atom.index for atom in atoms if atom.symbol != 'H']
    atoms = atoms[non_h_indices]
    num_atoms = len(atoms)
    print (f" Total number of atoms: {num_atoms}")

    # Define mapping of atoms into residues
    res_names = []
    res_numbers = []
    if "Tyr" in file:
        arg_limit = 7
        res_names += ['TYR']*arg_limit
    elif "Phe" in file:
        arg_limit = 6
        res_names += ['PHE']*arg_limit

    if "Arg" in file:
        res_names += ['ARG']*8
    elif "Lys" in file:
        res_names += ['LYS']*6

    res_names = np.array(res_names, dtype='U3')
    res_numbers = np.array([1] * arg_limit + [2] * (num_atoms - arg_limit), dtype=int)

    atoms.set_array('residuenumbers', res_numbers)
    atoms.set_array('residuenames', res_names, str)
    
    output_file = file.replace('.xyz', '.pdb')
    print (f" Writing to PDB file: {output_file}\n")
    write(output_file, atoms)

Reading from file structures/ArgTyr.xyz:
 Total number of atoms: 15
 Writing to PDB file: structures/ArgTyr.pdb

Reading from file structures/LysTyr.xyz:
 Total number of atoms: 13
 Writing to PDB file: structures/LysTyr.pdb

Reading from file structures/ArgTyrT.xyz:
 Total number of atoms: 15
 Writing to PDB file: structures/ArgTyrT.pdb

Reading from file structures/ArgTyrP.xyz:
 Total number of atoms: 15
 Writing to PDB file: structures/ArgTyrP.pdb

Reading from file structures/LysPhe.xyz:
 Total number of atoms: 12
 Writing to PDB file: structures/LysPhe.pdb

Reading from file structures/ArgPhe.xyz:
 Total number of atoms: 14
 Writing to PDB file: structures/ArgPhe.pdb

Reading from file structures/ArgPheT.xyz:
 Total number of atoms: 14
 Writing to PDB file: structures/ArgPheT.pdb



In [3]:
non_h_indices

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 24, 28]